In [ ]:
'''
Assisted with ChatGPT & the official examples from the M2M API documentation.
'''

import shutil

'''Setup entire script and define raw raster download'''
from utils.util import *
from utils.voice import notifySelf
import geopandas as gpd
import os
import sys
import time
import pandas as pd
import rasterio
import rioxarray
import socket
import traceback
import argparse

# # Create an argument parser
# parser = argparse.ArgumentParser(description="Process data for a range of years.")
#
# # Add arguments for startYear and endYear
# parser.add_argument(
#     "--startYear",
#     type=int,
#     required=True,
#     help="The starting year of the data range."
# )
# parser.add_argument(
#     "--endYear",
#     type=int,
#     required=True,
#     help="The ending year of the data range."
# )
# args = parser.parse_args()
# startYear, endYear = args.startYear, args.endYear

def gatherRawRasters(dataset, year, city, aoi_geodf):
    print(f'Gathering {dataset} for {year} in {city}.')
    if dataset == 'srtm_v3' and year != 2014:
        return
    bandNames = {'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'ST_B10', 'ST_EMIS', 'QA_PIXEL'}
    for month in range(1, 13):
        if month == 1:
            notifySelf(f'Starting on year {year} in dataset {dataset} as city {city}...')
        #Search for scenes
        print("Starting month", month)
        clear_folder(unprocessed_dir)
        search_payload = createSceneSearchPayload(dataset, aoi_geodf, year, month)
        scenes = sendRequest(serviceUrl + "scene-search", search_payload, apiKey)
        pd.json_normalize(scenes['results'])
        if len(scenes['results']) == 0:
            print("Month for scenes empty, skipping...")
            continue

        # Collect File IDs
        entityIds = [result['entityId'] for result in scenes['results'] if result['options']['bulk']]

        # Add to basket
        listId = f"{dataset}_{year}_{str(month)}_{socket.gethostname()}"
        scn_list_add_payload = {
            "listId": listId,
            'idField': 'entityId',
            "entityIds": entityIds,
            "datasetName": dataset
        }
        sendRequest(serviceUrl + "scene-list-add", scn_list_add_payload, apiKey)

        # Select URL download
        download_opt_payload = {
            "listId": listId,
            "datasetName": dataset,
        }
        products = sendRequest(serviceUrl + "download-options", download_opt_payload, apiKey)
        pd.json_normalize(products)

        # Collect File URLs based on product
        downloads = []
        if 'landsat_ot_c2_l2' in dataset:
            for product in products:
                if product["secondaryDownloads"]:
                    for secDownload in product["secondaryDownloads"]:
                        if secDownload["bulkAvailable"] and any(band in secDownload['displayId'] for band in bandNames):
                            downloads.append({"entityId": secDownload["entityId"], "productId": secDownload["id"]})
                        if secDownload['displayId'].endswith('_MTL.txt'):
                            downloads.append({"entityId": secDownload["entityId"], "productId": secDownload["id"]})
        elif 'srtm_v3' in dataset:
            for product in products:
                if product["bulkAvailable"] and product["entityId"] and product["id"]:
                    downloads.append({"entityId": product["entityId"], "productId": product["id"]})
        elif 'nlcd_collection_lndcov' in dataset:
            for product in products:
                if product["bulkAvailable"] and product["entityId"] and product["id"]:
                    downloads.append({"entityId": product["entityId"], "productId": product["id"]})
        download_req_payload = {
            "downloads": downloads,
            "label": listId
        }
        download_request_results = sendRequest(serviceUrl + "download-request", download_req_payload, apiKey)

        # Download Files Via URL
        if dataset == 'landsat_ot_c2_l2':
            results = download_request_results['availableDownloads']
            for result in results:
                runDownload(threads, result['url'])
        elif dataset == 'srtm_v3':
            results = download_request_results['preparingDownloads']
            for result in results:
                runDownload(threads, result['url'])
        else:
            results = download_request_results['preparingDownloads']
            for result in results:
                runDownload(threads, result['url'])
        for t in threads:
            t.join()
        for file in os.listdir(unprocessed_dir):
            if file.endswith('.tar'):
                try:
                    extract_specific_files(unprocessed_dir + '/' + file, unprocessed_dir)
                except:
                    print(f'Error: Could not extract file {file}')

        # Clear Basket
        remove_scnlst_payload = {"listId": listId}
        sendRequest(serviceUrl + "scene-list-remove", remove_scnlst_payload, apiKey)

        # Re-project Raster Files
        for tif in os.listdir(unprocessed_dir):
            if tif.endswith('.tif') or tif.endswith('.TIF'):
                temp_path = ".temp"  # Temporary file path
                input_path = unprocessed_dir + '/' + tif
                with rasterio.open(input_path) as src:
                    try:
                        color_map = src.colormap(1)
                    except ValueError:
                        color_map = None
                    with rioxarray.open_rasterio(input_path) as raster:
                        raster = raster.rio.reproject("EPSG:4326")
                        raster.rio.to_raster(temp_path, driver="GTiff")
                if color_map:
                    with rasterio.open(temp_path, "r+") as dst:
                        dst.write_colormap(1, color_map)
                os.replace(temp_path, input_path)
                if os.path.exists(temp_path):
                    os.remove(temp_path)

        # Move Unprocessed Files to Raw Folder
        for file in os.listdir(unprocessed_dir):
            if ".txt" in file or ".tif" in file or ".TIF" in file:
                if '1arc_v3' in file:
                    moveToRaw(file, 'DEM', f'{year}-01-01', city)
                    continue
                if 'Annual_NLCD' in file:
                    moveToRaw(file, 'Land_Cover', f'{year}-01-01', city)
                    continue
                date, band, coordinate = getMetaFromLandsatTIRs(file)
                if band in ['MTL', 'B10', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'EMIS', 'PIXEL']:
                    moveToRaw(file, 'oli', date, city)
        print('Finished moving')

        #You only need 1 month for these as they are annual
        if dataset == 'nlcd_collection_lndcov' or dataset == 'srtm_v3':
            break

    #Save progress
    with open('./Logs/raw_progress.txt', "a") as file:
        file.write(str(city) + ":" + str(year) + ":" + dataset + "\n")
    print('progress written for', city, year, dataset)

Logging in...


Login Successful, API Key Received!


In [2]:
datasets = ['landsat_ot_c2_l2', 'srtm_v3', 'nlcd_collection_lndcov']
years = [year for year in range(startYear, endYear+1)]

# Load city footprints from Esri Living Atlas
shapefile_folder = "./Data/area_shp/"
cities = []
aoi_geodfs = []
for file in os.listdir(shapefile_folder):
    if file.endswith(".shp"):
        cities.append(file.replace('Polygon_', '').replace('.shp', ''))
        aoi_geodf = gpd.read_file(shapefile_folder + file)
        aoi_geodf = aoi_geodf.to_crs("EPSG:4326")
        if aoi_geodf.empty:
            sys.exit("Error: Shapefile contains no data.")
        aoi_geodfs.append(aoi_geodf)
print("Shapefiles loaded successfully.")

for j, dataset in enumerate(datasets):
    for year in years:
        i = 0
        while i < int(len(cities)):
            try:
                clear_folder(unprocessed_dir)
                assert len(os.listdir(unprocessed_dir)) == 0, "Unprocessed directory is not empty."
                city, aoi_geodf = cities[i], aoi_geodfs[i]
                if os.path.exists('./Logs/raw_progress.txt'):
                    with open('./Logs/raw_progress.txt', 'r') as file:
                        progress = [line.split(':') for line in file.read().strip().split('\n')]
                    if any(city == instance[0] and str(year) == instance[1] and dataset == instance[2] for instance in progress):
                        print(f"{city}, {year}, {dataset} was gathered in the past.")
                    else:
                        gatherRawRasters(dataset, year, city, aoi_geodf)
                i +=1
            except Exception as e:
                print("An exception occurred:")
                print(f"Exception: {e}")
                notifySelf("An exception occurred:")
                notifySelf(f"Exception: {e}")
                traceback.print_exc()  # Print the full stack trace
                time.sleep(15)
print("Gathered raw rasters successfully.")
notifySelf("Gathered raw rasters successfully.")

KeyboardInterrupt: 

In [11]:
'''Merge all rasters to the first raster in the scene'''
shapefile_folder = "./Data/area_shp/"
cities_aoi = {} # Use cities dictionary instead of list to read from the raster itself
for file in os.listdir(shapefile_folder):
    if file.endswith(".shp"):
        aoi_geodf = gpd.read_file(shapefile_folder + file)
        aoi_geodf = aoi_geodf.to_crs("EPSG:4326")
        if aoi_geodf.empty:
            sys.exit("Error: Shapefile contains no data.")
        cities_aoi[file.replace('Polygon_', '').replace('.shp', '')] = aoi_geodf

import rasterio
import rasterio.merge

def merge_rasters(target_file, scene_files):
    """Merge multiple raster files into the first raster in the scene."""
    src_files_to_merge = []

    for scene_file in scene_files:
        src = rasterio.open(scene_file)
        src_files_to_merge.append(src)

    mosaic, out_trans = rasterio.merge.merge(src_files_to_merge)

    out_meta = src_files_to_merge[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans
    })

    with rasterio.open(target_file, "w", **out_meta) as dest:
        dest.write(mosaic)

    for src in src_files_to_merge:
        src.close()


def checkRasterInPolygonAtAll(polygon: GeoDataFrame, ras: str) -> bool:
    polygon = polygon.geometry.iloc[0]
    with rasterio.open(ras) as src:
        bounds = src.bounds
        raster_bounds = Polygon([
            (bounds.left, bounds.top),
            (bounds.right, bounds.top),
            (bounds.right, bounds.bottom),
            (bounds.left, bounds.bottom),
            (bounds.left, bounds.top)
        ])

    return polygon.intersects(raster_bounds)

def list_files_in_folder(folder_path):
    files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if
             os.path.isfile(os.path.join(folder_path, f))]
    return files
notifySelf("Starting merge of DEM and Land_Cover...")
allGeoFiles = get_file_paths(raw_dir)
allDEMLandCover, cityDatesTaken = [], []
for geoFile in tqdm(allGeoFiles, desc="Filtering only 1st DEM,LC in scenes..."):
    geoFileParts = geoFile.split('/')
    fileName, date, city, dataType = geoFileParts[-1], geoFileParts[-2], geoFileParts[-3], geoFileParts[-4]
    polygon = cities_aoi[city]
    if dataType == "DEM":              '''or dataType == "Land_Cover"'''
        if city + "-" + date + "-" + dataType in cityDatesTaken:
            continue
        if checkRasterInPolygonAtAll(polygon, geoFile):
            cityDatesTaken.append(city + "-" + date + "-" + dataType)
            allDEMLandCover.append(geoFile)
i, file25Percent = 0, int(len(allDEMLandCover)//4)
for dem_lc_file in tqdm(allDEMLandCover, desc="Merging DEM and Land_Cover"):
    if i % file25Percent == 0:
        percentage_done = int((i / len(allGeoFiles)) * 100)
        notifySelf(f'We are at {percentage_done}% merged DEM LC')
    i+=1
    geoFileParts = dem_lc_file.split('/')
    fileName, date, city, dataType = geoFileParts[-1], geoFileParts[-2], geoFileParts[-3], geoFileParts[-4]
    targetFile = os.path.join(raw_dir, dataType, city, date, "Merged_" + fileName)
    polygon = cities_aoi[city]
    dem_lc_scene_files = list_files_in_folder(os.path.dirname(os.path.abspath(dem_lc_file)))
    if len(dem_lc_scene_files) > 1:
        print("Merging to " + targetFile)
        merge_rasters(targetFile, dem_lc_scene_files)

Merging DEM and Land_Cover:   0%|          | 0/107 [00:00<?, ?it/s]

Merging to ./Temp/RawRasters/DEM/Los_Angeles_CA/2014-01/Merged_n34_w119_1arc_v3.tif


Merging DEM and Land_Cover:   1%|          | 1/107 [00:01<03:28,  1.97s/it]

Merging to ./Temp/RawRasters/DEM/Lexington-Fayette_KY/2014-01/Merged_n37_w085_1arc_v3.tif


Merging DEM and Land_Cover:   5%|▍         | 5/107 [00:02<00:39,  2.56it/s]

...

Merging DEM and Land_Cover:  98%|█████████▊| 105/107 [00:28<00:00,  2.56it/s]

Merging to ./Temp/RawRasters/DEM/Knoxville_TN/2014-01/Merged_n36_w085_1arc_v3.tif


Merging DEM and Land_Cover:  99%|█████████▉| 106/107 [00:29<00:00,  2.14it/s]

Merging to ./Temp/RawRasters/DEM/Pahrump_NV/2014-01/Merged_n36_w116_1arc_v3.tif


Merging DEM and Land_Cover: 100%|██████████| 107/107 [00:30<00:00,  3.56it/s]


In [12]:
'''Clip Raster, we are reading metadata from rastername'''
notifySelf("Starting clip list of raster paths...")
allGeoFiles = get_file_paths(raw_dir)
i, file25Percent = 0, int(len(allGeoFiles)//4)
usableTIFFs = []
for geoFilePath in tqdm(allGeoFiles, desc="Make raster list/move txt files"):
    if i % file25Percent == 0:
        percentage_done = int((i / len(allGeoFiles)) * 100)
        notifySelf(f'We are at {percentage_done}% clipped')
    i+=1
    geoFileParts = geoFilePath.split('/')
    fileName, date, city, dataType = geoFileParts[-1], geoFileParts[-2], geoFileParts[-3], geoFileParts[-4]
    targetFile = os.path.join(clipped_dir, dataType, city, date, fileName)
    if os.path.exists(targetFile):
        continue
    polygon = cities_aoi[city]
    if geoFilePath.endswith(".tif") or geoFilePath.endswith(".TIF"):
        if checkPolygonInRasterCompletely(polygon, geoFilePath):
            usableTIFFs.append(geoFilePath)
    elif geoFilePath.endswith(".txt"):
        moveToClipped(geoFilePath, fileName, dataType, date, city)

Make raster list/move txt files: 100%|██████████| 62027/62027 [1:19:42<00:00, 12.97it/s]  


In [13]:
save_paths_to_log(usableTIFFs)

Saving to log: 100%|██████████| 44/44 [00:00<00:00, 468399.43it/s]

File paths saved to ./Logs/clipped_processed.txt


In [14]:
'''Clip and move Rasters'''
filesList = read_file_paths_from_log()
i, file25Percent = 0, int(len(filesList) // 4)
notifySelf("Starting clip Rasters...")
#Clip, project and move to new folder.
for geoFilePath in tqdm(filesList, desc="Clipping rasters"):
    if i % file25Percent == 0:
        percentage_done = int((i / len(filesList)) * 100)
        notifySelf(f'We are at {percentage_done}% clipped')
    i += 1
    geoFileParts = geoFilePath.split('/')
    fileName, date, city, dataType = geoFileParts[-1], geoFileParts[-2], geoFileParts[-3], geoFileParts[-4]
    target_folder = os.path.join(clipped_dir, dataType, city, date)
    os.makedirs(target_folder, exist_ok=True)
    target_file = os.path.join(clipped_dir, dataType, city, date, fileName)
    if os.path.exists(target_file):
        continue
    polygon = cities_aoi[city]
    raster = rioxarray.open_rasterio(geoFilePath)
    raster_reprojected = raster.rio.reproject("EPSG:4326")
    colormap = None
    with rasterio.open(geoFilePath) as src:
        if src.colorinterp[0] == rasterio.enums.ColorInterp.palette:
            colormap = src.colormap(1)  # Assuming band 1 has the colormap
    clipped = raster_reprojected.rio.clip(polygon.geometry, polygon.crs, drop=True)
    clipped.rio.to_raster(target_file)
    if colormap:
        with rasterio.open(target_file, "r+") as dest:
            dest.write_colormap(1, colormap)
    raster.close()
    raster_reprojected.close()
print("Clipped and moved raw rasters successfully.")
notifySelf("Clipped and moved raw rasters successfully.")

Clipping rasters: 100%|██████████| 58296/58296 [01:12<00:00, 800.84it/s] 


Clipped and moved raw rasters successfully.


In [36]:
'''Categorize rasters by cloud coverage'''
def moveToProcess(filePath: str, fileName: str, typeFolder: str, date: str, city: str, cloud_cover: str):
    target_folder = os.path.join(process_dir, typeFolder, cloud_cover, city, date)
    os.makedirs(target_folder, exist_ok=True)
    target_file_path = os.path.join(target_folder, fileName)
    if os.path.exists(target_file_path):
        return
    shutil.copy2(filePath, target_file_path)

notifySelf("Starting cloud removal...")
process_dir = temp_dir + '/Process'
allClippedFiles, validCloudFiles, cloudRange = [], [], [5, 15, 25, 40, 60, 80, 100]
for clippedFilePath in get_file_paths(clipped_dir):
    if 'QA_PIXEL' in clippedFilePath:
        allClippedFiles.append(clippedFilePath)
i, file25Percent = 0, int(len(allClippedFiles) // 4)
for clippedFilePath in tqdm(allClippedFiles, desc="Filtering Clouds & Missing Tifs"):
    if i % file25Percent == 0:
        percentage_done = int((i / len(allClippedFiles)) * 100)
        notifySelf(f'We are at {percentage_done}% clipped')
    i += 1
    cloudCover, cloudCategory = calculate_cloud_cover_percentage(clippedFilePath), 0
    for coverPercentage in cloudRange:
        if cloudCover <= coverPercentage:
            cloudCategory = coverPercentage
            break;
    cloudCategory = f'less{cloudCategory}CloudCover'
    fileParts = clippedFilePath.split('/')
    fileName, date, city, dataType = fileParts[-1], fileParts[-2], fileParts[-3], fileParts[-4]
    sceneIdentifyingName = '_'.join(fileName.split('_')[:7])
    sceneFiles = []
    for sceneFileAbsolutePath in list_files_in_folder(os.path.dirname(os.path.abspath(clippedFilePath))):
        if sceneIdentifyingName in sceneFileAbsolutePath:
            sceneFiles.append(sceneFileAbsolutePath)
    if len(sceneFiles) == 10:
        for sceneFile in sceneFiles:
            moveToProcess(sceneFile, sceneFile.split('/')[-1], dataType, date, city, cloudCategory)

Filtering Clouds & Missing Tifs: 100%|██████████| 24155/24155 [1:42:11<00:00,  3.94it/s]  


In [ ]:
'''Process indices'''

def list_files_in_folder(folder_path):
    files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if
             os.path.isfile(os.path.join(folder_path, f))]
    return files


def extract_constants(mtl_file_path):
    constants = {}
    with open(mtl_file_path, 'r') as mtl_file:
        lines = mtl_file.readlines()
    for line in lines:
        line = line.strip()
        if line.startswith("RADIANCE_MULT_BAND_10"):
            constants['ML'] = float(line.split(" = ")[1])
        elif line.startswith("TEMPERATURE_ADD_BAND_ST_B10"):
            constants['ST_OFFSET'] = float(line.split(" = ")[1])
        elif line.startswith("TEMPERATURE_MULT_BAND_ST_B10"):
            constants['ST_MULT'] = float(line.split(" = ")[1])
        elif line.startswith("RADIANCE_ADD_BAND_10"):
            constants['AL'] = float(line.split(" = ")[1])
        elif line.startswith("K1_CONSTANT_BAND_10"):
            constants['K1'] = float(line.split(" = ")[1])
        elif line.startswith("K2_CONSTANT_BAND_10"):
            constants['K2'] = float(line.split(" = ")[1])
        for band in range(2, 7):  # Loop through bands 2 to 6
            mult_key = f"REFLECTANCE_MULT_BAND_{band}"
            add_key = f"REFLECTANCE_ADD_BAND_{band}"
            if line.startswith(mult_key):
                constants[mult_key] = float(line.split(" = ")[1])
            elif line.startswith(add_key):
                constants[add_key] = float(line.split(" = ")[1])
    return constants


def retrieve_ndwi(green_band_path, nir_band_path, reflectance_mult: dict, reflectance_add: dict, output_path: str):
    with rasterio.open(green_band_path) as green_src, rasterio.open(nir_band_path) as nir_src, \
         rasterio.open(cloud) as cloud_src:
        green = green_src.read(1).astype('float32') * reflectance_mult['B3'] + reflectance_add['B3']
        nir = nir_src.read(1).astype('float32') * reflectance_mult['B5'] + reflectance_add['B5']
        qa_pixel = cloud_src.read(1).astype('uint16')
        cloud_nodata_value = cloud_src.nodata
        nodata_value = nir_src.nodata
        if cloud_nodata_value is not None:
            cloud_nodata_mask = (qa_pixel == cloud_nodata_value)
        else:
            cloud_nodata_mask = np.zeros_like(qa_pixel, dtype=bool)
        cloud_confidence_mask = (qa_pixel & 0b00011000) >> 3
        cloud_shadow_mask = (qa_pixel & 0b00100000) >> 5
        cloud_pixels = (cloud_confidence_mask >= 1) | (cloud_shadow_mask == 1)
        valid_pixels = ~cloud_nodata_mask
        cloud_mask = cloud_pixels & valid_pixels
        green = np.clip(green, 0, 1)
        nir = np.clip(nir, 0, 1)

        ndwi = np.where((green + nir) == 0, 0, (green - nir) / (green + nir))
        ndwi[cloud_mask] = nodata_value

        profile = green_src.profile
        profile.update(dtype=rasterio.float32, count=1)

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(ndwi, 1)


def retrieve_ndvi(nir_band_path, red_band_path, cloud, reflectance_mult: dict, reflectance_add: dict, output_path):
    with rasterio.open(nir_band_path) as nir_src, rasterio.open(red_band_path) as red_src, \
         rasterio.open(cloud) as cloud_src:
        nir = nir_src.read(1).astype('float32') * reflectance_mult['B5'] + reflectance_add['B5']
        red = red_src.read(1).astype('float32') * reflectance_mult['B4'] + reflectance_add['B4']
        qa_pixel = cloud_src.read(1).astype('uint16')
        cloud_nodata_value = cloud_src.nodata
        nodata_value = red_src.nodata
        if cloud_nodata_value is not None:
            cloud_nodata_mask = (qa_pixel == cloud_nodata_value)
        else:
            cloud_nodata_mask = np.zeros_like(qa_pixel, dtype=bool)
        cloud_confidence_mask = (qa_pixel & 0b00011000) >> 3
        cloud_shadow_mask = (qa_pixel & 0b00100000) >> 5
        cloud_pixels = (cloud_confidence_mask >= 1) | (cloud_shadow_mask == 1)
        valid_pixels = ~cloud_nodata_mask
        cloud_mask = cloud_pixels & valid_pixels
        nir = np.clip(nir, 0, 1)
        red = np.clip(red, 0, 1)
        ndvi = np.where((nir + red) == 0, 0, (nir - red) / (nir + red))
        ndvi[cloud_mask] = nodata_value
        profile = nir_src.profile
        profile.update(dtype=rasterio.float32, count=1)
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(ndvi, 1)

def retrieve_ndbi(swir1_band_path, nir_band_path, cloud, reflectance_mult: dict, reflectance_add: dict, output_path):
    with rasterio.open(swir1_band_path) as swir1_src, rasterio.open(nir_band_path) as nir_src, \
         rasterio.open(cloud) as cloud_src:
        # Read bands and apply reflectance scaling
        swir1 = swir1_src.read(1).astype('float32') * reflectance_mult['B6'] + reflectance_add['B6']
        nir = nir_src.read(1).astype('float32') * reflectance_mult['B5'] + reflectance_add['B5']
        
        # Read QA pixel data for cloud masking
        qa_pixel = cloud_src.read(1).astype('uint16')
        cloud_nodata_value = cloud_src.nodata
        nodata_value = nir_src.nodata
        
        # Create cloud mask
        if cloud_nodata_value is not None:
            cloud_nodata_mask = (qa_pixel == cloud_nodata_value)
        else:
            cloud_nodata_mask = np.zeros_like(qa_pixel, dtype=bool)
            
        # Extract cloud confidence and cloud shadow bits
        cloud_confidence_mask = (qa_pixel & 0b00011000) >> 3
        cloud_shadow_mask = (qa_pixel & 0b00100000) >> 5
        
        # A pixel is flagged as cloudy if cloud confidence is >= 1 or if cloud shadow bit is set
        cloud_pixels = (cloud_confidence_mask >= 1) | (cloud_shadow_mask == 1)
        
        # Combine with valid pixels
        valid_pixels = ~cloud_nodata_mask
        cloud_mask = cloud_pixels & valid_pixels
        
        # Clip reflectance values to valid range [0,1]
        swir1 = np.clip(swir1, 0, 1)
        nir = np.clip(nir, 0, 1)
        
        # Calculate NDBI
        ndbi = np.where((swir1 + nir) == 0, 0, (swir1 - nir) / (swir1 + nir))
        
        # Apply cloud mask
        ndbi[cloud_mask] = nodata_value
        
        # Update profile for output
        profile = swir1_src.profile
        profile.update(dtype=rasterio.float32, count=1)
        
        # Write the resulting NDBI raster to disk
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(ndbi, 1)

def retrieve_lst(band10_path, cloud, constants, output_path):
    with rasterio.open(band10_path) as band10_src, \
         rasterio.open(cloud) as cloud_src:

        # Read raster data
        q_cal = band10_src.read(1).astype('float32')
        qa_pixel = cloud_src.read(1).astype('uint16')

        # Get nodata values
        band10_nodata = band10_src.nodata
        cloud_nodata_value = cloud_src.nodata  # Get NoData value from raster metadata

        # Valid mask to exclude nodata values
        valid_mask = (q_cal != band10_nodata)

        if cloud_nodata_value is not None:
            cloud_nodata_mask = (qa_pixel == cloud_nodata_value)
        else:
            cloud_nodata_mask = np.zeros_like(qa_pixel, dtype=bool)

        cloud_confidence_mask = (qa_pixel & 0b00011000) >> 3
        cloud_shadow_mask = (qa_pixel & 0b00100000) >> 5
        cloud_pixels = (cloud_confidence_mask >= 1) | (cloud_shadow_mask == 1)

        valid_pixels = ~cloud_nodata_mask
        cloud_mask = cloud_pixels & valid_pixels

        valid_mask = valid_mask & ~cloud_mask

        # Apply scale factor and offset to get temperature in Kelvin
        T_K = (q_cal * constants['ST_MULT']) + constants['ST_OFFSET']

        # Convert Kelvin to Fahrenheit
        lst_fahrenheit = np.where(valid_mask, (T_K - 273.15) * 1.8 + 32, np.nan)

        # Save the new raster
        with rasterio.open(
            output_path,
            "w",
            driver="GTiff",
            height=band10_src.height,
            width=band10_src.width,
            count=1,
            dtype=lst_fahrenheit.dtype,
            crs=band10_src.crs,
            transform=band10_src.transform,
            nodata=np.nan
        ) as dst:
            dst.write(lst_fahrenheit, 1)

    # print(f"Converted LST saved to: {output_path}")






def retrieve_albedo_all(band2_path, band3_path, band4_path, band5_path, band6_path, qa_path,
                        reflectance_mult: dict, reflectance_add: dict, output_path):
    coefficients = {
        "band2": 0.356,
        "band3": 0.130,
        "band4": 0.373,
        "band5": 0.085,
        "band6": 0.072,
        "offset": -0.018
    }

    with rasterio.open(band2_path) as band2_src, \
         rasterio.open(band3_path) as band3_src, \
         rasterio.open(band4_path) as band4_src, \
         rasterio.open(band5_path) as band5_src, \
         rasterio.open(band6_path) as band6_src, \
         rasterio.open(qa_path) as cloud_src:  # QA band is opened but not used here

        # Read raw bands (before applying scaling)
        raw_band2 = band2_src.read(1).astype('float32')
        raw_band3 = band3_src.read(1).astype('float32')
        raw_band4 = band4_src.read(1).astype('float32')
        raw_band5 = band5_src.read(1).astype('float32')
        raw_band6 = band6_src.read(1).astype('float32')
        qa_pixel = cloud_src.read(1).astype('uint16')

        # Get the nodata value (assumes the same nodata for all bands)
        nodata_value = band2_src.nodata
        cloud_nodata_value = cloud_src.nodata

        # Create a nodata mask from the raw bands
        nodata_mask = ((raw_band2 == nodata_value) |
                       (raw_band3 == nodata_value) |
                       (raw_band4 == nodata_value) |
                       (raw_band5 == nodata_value) |
                       (raw_band6 == nodata_value))

        # Apply scaling to each band using the provided reflectance multipliers and offsets
        band2 = raw_band2 * reflectance_mult['B2'] + reflectance_add['B2']
        band3 = raw_band3 * reflectance_mult['B3'] + reflectance_add['B3']
        band4 = raw_band4 * reflectance_mult['B4'] + reflectance_add['B4']
        band5 = raw_band5 * reflectance_mult['B5'] + reflectance_add['B5']
        band6 = raw_band6 * reflectance_mult['B6'] + reflectance_add['B6']

        # Compute albedo using the NTB coefficients (clip each band between 0 and 1)
        albedo = (
            coefficients["band2"] * np.clip(band2, 0, 1) +
            coefficients["band3"] * np.clip(band3, 0, 1) +
            coefficients["band4"] * np.clip(band4, 0, 1) +
            coefficients["band5"] * np.clip(band5, 0, 1) +
            coefficients["band6"] * np.clip(band6, 0, 1) +
            coefficients["offset"]
        )

        if cloud_nodata_value is not None:
            cloud_nodata_mask = (qa_pixel == cloud_nodata_value)
        else:
            cloud_nodata_mask = np.zeros_like(qa_pixel, dtype=bool)
        cloud_confidence_mask = (qa_pixel & 0b00011000) >> 3
        cloud_shadow_mask = (qa_pixel & 0b00100000) >> 5
        cloud_pixels = (cloud_confidence_mask >= 1) | (cloud_shadow_mask == 1)

        valid_pixels = ~cloud_nodata_mask
        cloud_mask = cloud_pixels & valid_pixels

        # Apply the nodata mask to the computed albedo: any pixel with nodata in any band is set to nodata.
        albedo[nodata_mask | cloud_mask] = nodata_value


        # Update the output profile to include the nodata setting and ensure the dtype is correct
        profile = band2_src.profile
        profile.update(dtype=rasterio.float32, count=1, nodata=nodata_value)

        # Write the resulting albedo raster to disk
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(albedo, 1)

def retrieve_albedo(band2_path, band3_path, band4_path, band5_path, band6_path, qa_path,
                    reflectance_mult: dict, reflectance_add: dict, output_path):
    """
    Compute surface albedo from Landsat 8 bands and exclude cloudy pixels following
    the logic used in calculate_cloud_cover_percentage.

    The albedo is computed as:

    albedo = c2 * clip(B2, 0, 1) + c3 * clip(B3, 0, 1) + c4 * clip(B4, 0, 1) +
             c5 * clip(B5, 0, 1) + c6 * clip(B6, 0, 1) + offset

    In LaTeX:

    ```latex
    \[
      \text{albedo} = 0.356 \cdot \text{clip}(B_2, 0, 1)
                    + 0.130 \cdot \text{clip}(B_3, 0, 1)
                    + 0.373 \cdot \text{clip}(B_4, 0, 1)
                    + 0.085 \cdot \text{clip}(B_5, 0, 1)
                    + 0.072 \cdot \text{clip}(B_6, 0, 1)
                    - 0.018
    \]
    ```

    Cloudy pixels are determined by:
      - cloud_confidence_mask = (QA_PIXEL & 0b00011000) >> 3
      - cloud_shadow_mask   = (QA_PIXEL & 0b00100000) >> 5

    A pixel is flagged as cloudy if:

    ```python
    cloud_mask = (cloud_confidence_mask >= 1) | (cloud_shadow_mask == 1)
    ```

    These cloudy pixels, along with any pixels already marked as nodata,
    are then set to the nodata value in the final albedo raster.
    """
    # NTB coefficients for Landsat 8 bands
    coefficients = {
        "band2": 0.356,
        "band3": 0.130,
        "band4": 0.373,
        "band5": 0.085,
        "band6": 0.072,
        "offset": -0.018
    }

    with rasterio.open(band2_path) as band2_src, \
         rasterio.open(band3_path) as band3_src, \
         rasterio.open(band4_path) as band4_src, \
         rasterio.open(band5_path) as band5_src, \
         rasterio.open(band6_path) as band6_src, \
         rasterio.open(qa_path) as qa_src:

        # Read raw bands (before applying scaling)
        raw_band2 = band2_src.read(1).astype('float32')
        raw_band3 = band3_src.read(1).astype('float32')
        raw_band4 = band4_src.read(1).astype('float32')
        raw_band5 = band5_src.read(1).astype('float32')
        raw_band6 = band6_src.read(1).astype('float32')

        # Get the nodata value (assumes the same nodata for all bands)
        nodata_value = band2_src.nodata

        # Create a nodata mask from the raw bands
        nodata_mask = ((raw_band2 == nodata_value) |
                       (raw_band3 == nodata_value) |
                       (raw_band4 == nodata_value) |
                       (raw_band5 == nodata_value) |
                       (raw_band6 == nodata_value))

        # Apply scaling to each band using the provided reflectance multipliers and offsets
        band2 = raw_band2 * reflectance_mult['B2'] + reflectance_add['B2']
        band3 = raw_band3 * reflectance_mult['B3'] + reflectance_add['B3']
        band4 = raw_band4 * reflectance_mult['B4'] + reflectance_add['B4']
        band5 = raw_band5 * reflectance_mult['B5'] + reflectance_add['B5']
        band6 = raw_band6 * reflectance_mult['B6'] + reflectance_add['B6']

        # Compute albedo using the NTB coefficients (clipping values between 0 and 1)
        albedo = (
            coefficients["band2"] * np.clip(band2, 0, 1) +
            coefficients["band3"] * np.clip(band3, 0, 1) +
            coefficients["band4"] * np.clip(band4, 0, 1) +
            coefficients["band5"] * np.clip(band5, 0, 1) +
            coefficients["band6"] * np.clip(band6, 0, 1) +
            coefficients["offset"]
        )

        # --- Cloud Masking using the QA_PIXEL band ---
        # Read the QA_PIXEL band
        qa = qa_src.read(1)

        # Extract cloud confidence and cloud shadow bits using the same logic as in
        # calculate_cloud_cover_percentage
        cloud_confidence_mask = (qa & 0b00011000) >> 3
        cloud_shadow_mask = (qa & 0b00100000) >> 5

        # A pixel is flagged as cloudy if cloud confidence is >= 1 or if cloud shadow bit is set
        cloud_mask = (cloud_confidence_mask >= 1) | (cloud_shadow_mask == 1)

        # Combine the original nodata mask with the cloud mask
        final_mask = nodata_mask | cloud_mask

        # Set albedo to nodata_value for pixels flagged as nodata or cloudy
        albedo[final_mask] = nodata_value

        # Update the output profile, ensuring nodata is preserved and the dtype is correct
        profile = band2_src.profile
        profile.update(dtype=rasterio.float32, count=1, nodata=nodata_value)

        # Write the resulting albedo raster to disk
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(albedo, 1)


def getDataPath(fileName, typeFolder: str, date, city, cloud_cover):
    target_folder = os.path.join(data_dir, typeFolder, cloud_cover, city, date)
    os.makedirs(target_folder, exist_ok=True)
    target_file_path = os.path.join(target_folder, fileName)
    return target_file_path


def getClippedPath(typeFolder: str, date, city):
    target_folder = os.path.join(clipped_dir, typeFolder, city, date)
    if os.path.exists(target_folder):
        target_files = os.listdir(target_folder)
        if len(target_files) > 0:
            targetFile = os.listdir(target_folder)[0]
            return os.path.join(target_folder, targetFile)
    return 'invalid'

def raster_statistics(raster_path):
    with rasterio.open(raster_path) as dataset:
        band1 = dataset.read(1)  # Read the first band

        # Mask out no-data values if they exist
        if dataset.nodata is not None:
            valid_mask = band1 != dataset.nodata
            band1 = band1[valid_mask]

        min_val = np.min(band1)
        max_val = np.max(band1)
        mean_val = np.mean(band1)
        print(band1.meta)
        print(band1.scales, src.offsets)
        print(f'Band10 Mean value: {mean_val}, max: {max_val}, min: {min_val}')

def create_heat_index(lst_path, output_path):
    with rasterio.open(lst_path) as src:
        lst = src.read(1).astype('float32')  # Ensure float32 for NaN support
        nodata_value = src.nodata  # Get original NoData value (NaN or a specific number)

        # Identify valid pixels
        valid_mask = ~np.isnan(lst) if np.isnan(nodata_value) else lst != nodata_value
        valid_lst = lst[valid_mask]

        # If there are no valid pixels, raise an error
        if valid_lst.size == 0:
            raise ValueError("No valid LST data found. Check input raster.")

        # Determine min and max for valid LST values
        lst_min, lst_max = np.min(valid_lst), np.max(valid_lst)

        # Define 10 categories for the heat index
        category_bounds = np.linspace(lst_min, lst_max, 11)

        # Initialize the heat index raster with NaN (to preserve NoData)
        lst_categories = np.full_like(lst, np.nan, dtype='float32')

        # Assign categories only to valid pixels
        lst_categories[valid_mask] = np.digitize(valid_lst, category_bounds, right=True)

        # Preserve the profile and update metadata
        profile = src.profile
        profile.update(dtype=rasterio.float32, count=1, nodata=nodata_value)  # Keep nodata consistent

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(lst_categories, 1)

def get_file_paths(folder_path):
    file_paths = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            full_path = os.path.abspath(os.path.join(root, file))
            file_paths.append(full_path)
    return file_paths

allQAPixelFiles = []
for filePath in get_file_paths(process_dir):
    if 'QA_PIXEL' in filePath:
        allQAPixelFiles.append(filePath)
for filePath in tqdm(allQAPixelFiles, desc="Processing"):
    fileParts = filePath.split('/')
    fileName, date, city, cloudCategory, dataType = fileParts[-1], fileParts[-2], fileParts[-3], fileParts[-4], fileParts[-5]
    sceneIdentifyingName = '_'.join(fileName.split('_')[:7])
    sceneFiles = []
    for sceneFileAbsolutePath in list_files_in_folder(os.path.dirname(os.path.abspath(filePath))):
        if sceneIdentifyingName in sceneFileAbsolutePath:
            sceneFiles.append(sceneFileAbsolutePath)
    band_files = {}

    for sceneFile in sceneFiles:
        band = sceneFile.split('_')[-1].replace('.TIF', '').replace('.txt', '').replace('.tif', '')
        band_files[band] = sceneFile  # Store all band files in a dictionary

    for sceneFile in sceneFiles:
        band = sceneFile.split('_')[-1].replace('.TIF', '').replace('.txt', '').replace('.tif', '')
        if band == 'MTL':
            MTL = sceneFile
        if band == 'PIXEL':
            cloud = sceneFile
    constants = extract_constants(MTL)

    for sceneFile in sceneFiles:
        band = sceneFile.split('_')[-1].replace('.TIF', '').replace('.txt', '').replace('.tif', '')
        if band == 'MTL':
            MTL = sceneFile
        if band == 'PIXEL':
            cloud = sceneFile
        if band == 'B10':
            band10 = sceneFile
        if band == 'B2':
            band2 = sceneFile
        if band == 'B3':
            band3 = sceneFile
        if band == 'B4':
            band4 = sceneFile
        if band == 'B5':
            band5 = sceneFile
        if band == 'B6':
            band6 = sceneFile
        if band == 'B7':
            band7 = sceneFile
        if band == 'EMIS':
            emis = sceneFile
    constants = extract_constants(MTL)
    reflectance_mult, reflectance_add = {}, {}
    bands = ["B2", "B3", "B4", "B5", "B6"]
    for band in bands:
        reflectance_mult[band] = constants[f"REFLECTANCE_MULT_BAND_{band.replace('B', '')}"]
        reflectance_add[band] = constants[f"REFLECTANCE_ADD_BAND_{band.replace('B', '')}"]
    # raster_statistics(band10)
    # print(calculate_cloud_cover_percentage(cloud))
    demPath = getClippedPath('DEM', '2014-01', city)
    landCover = getClippedPath('Land_Cover', date.split('-')[0] + "-01", city)
    if os.path.exists(demPath) and os.path.exists(landCover):
        # shutil.copy2(demPath, getDataPath('DEM.tif', 'X', date, city, cloudCategory))
        # shutil.copy2(landCover, getDataPath('Land_Cover.tif', 'X', date, city, cloudCategory))
        retrieve_ndbi(band6, band5, cloud, reflectance_mult, reflectance_add, getDataPath('NDBI.tif', 'X', date, city, cloudCategory))
        # retrieve_ndvi(band5, band4, cloud, reflectance_mult, reflectance_add, getDataPath('NDVI.tif', 'X', date, city, cloudCategory))
        # retrieve_ndwi(band3, band5, reflectance_mult, reflectance_add, getDataPath('NDWI.tif', 'X', date, city, cloudCategory))
        # retrieve_albedo_all(band2, band3, band4, band5, band6, cloud, reflectance_mult, reflectance_add, getDataPath('Albedo.tif', 'X', date, city, cloudCategory))
        # retrieve_lst(band10, cloud, constants, getDataPath('LST.tif', 'y', date, city, cloudCategory))
        # create_heat_index(getDataPath('LST.tif', 'y', date, city, cloudCategory), getDataPath('Heat_Index.tif', 'y', date, city, cloudCategory))

Processing:   0%|          | 0/16284 [00:00<?, ?it/s]/work/ubh496/podman_storage/ipykernel_1916834/27504549.py:86: RuntimeWarning: invalid value encountered in divide
  ndvi = np.where((nir + red) == 0, 0, (nir - red) / (nir + red))
/work/ubh496/podman_storage/ipykernel_1916834/27504549.py:57: RuntimeWarning: invalid value encountered in divide
  ndwi = np.where((green + nir) == 0, 0, (green - nir) / (green + nir))
Processing: 100%|██████████| 16284/16284 [1:38:52<00:00,  2.74it/s] 


In [10]:
deg_to_m = 111320
for raster_path in ["/work/ubh496/heat-island-test/Data/X/less5CloudCover/Abilene_TX/2013-06/Albedo.tif", "/work/ubh496/heat-island-test/Data/X/less5CloudCover/Abilene_TX/2013-06/DEM.tif", "/work/ubh496/heat-island-test/Data/X/less5CloudCover/Abilene_TX/2013-06/Land_Cover.tif", "/work/ubh496/heat-island-test/Data/X/less5CloudCover/Abilene_TX/2013-06/NDVI.tif", "/work/ubh496/heat-island-test/Data/X/less5CloudCover/Abilene_TX/2013-06/NDWI.tif", "/work/ubh496/heat-island-test/Data/y/less5CloudCover/Abilene_TX/2013-06/LST.tif"]:
    with rasterio.open(raster_path) as dataset:
        pixel_width_deg, pixel_height_deg = dataset.res
        pixel_width_m = pixel_width_deg * deg_to_m
        pixel_height_m = pixel_height_deg * deg_to_m
        print(raster_path.split('/')[-1] + " " + str(pixel_width_m) + " " + str(pixel_height_m))  # (pixel width, pixel height)

Albedo.tif 33.22878972943077 33.22878972943113
DEM.tif 30.922222222220395 30.922222222222047
Land_Cover.tif 33.077629425293466 33.077629425293836
NDVI.tif 33.22878972943077 33.22878972943113
NDWI.tif 33.22878972943077 33.22878972943113
LST.tif 33.22878972943077 33.22878972943113
